In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib datasets

%pip install nltk
%pip install contractions
%pip install spacy
%pip install SentencePiece 
%pip install transformers==4.26.1

In [58]:
import pandas as pd
import seaborn as sns
import numpy as np
import nltk
nltk.download('punkt')


     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 33.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/240.0 kB ? eta -:--:--
   --------------------------------------- 240.0/240.0 kB 14.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/240.7 kB ? eta -:--:--
   --------------------------------------- 240.7/240.7 kB 14.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/87.5 kB ? eta -:--:--
   ---------------------------------------- 87.5/87.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [ ]:
import os
import matplotlib.pyplot as plt
import sklearn

import contractions
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stop_words = set(stopwords.words('english'))

from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

import spacy
nlp = spacy.load("en_core_web_sm")

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

import keras
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding, Input

In [66]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd


print("Libraries imported")

Libraries imported


In [3]:
(train_ds, test_ds, val_ds), metadata = tfds.load('huggingface:iwslt2017/iwslt2017-en-zh', split=['train', 'test', 'validation'], with_info=True)

full_ds = train_ds.concatenate(test_ds).concatenate(val_ds)

print("Data Structure:")
print(metadata.features)

C:\Users\kelvi\lib\site-packages\tensorflow_datasets\core\dataset_builders\huggingface_dataset_builder.py:160: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  hf_names = hf_datasets.list_datasets()


Data Structure:
FeaturesDict({
    'translation': Translation({
        'en': Text(shape=(), dtype=string),
        'zh': Text(shape=(), dtype=string),
    }),
})


In [4]:
print("%s%s" % ("Train: ", train_ds.cardinality().numpy()))
print("%s%s" % ("Test: ", test_ds.cardinality().numpy()))
print("%s%s" % ("Validation: ", val_ds.cardinality().numpy()))
print("%s%s" % ("Total: ", full_ds.cardinality().numpy()))

Train: 231266
Test: 8549
Validation: 879
Total: 240694


In [12]:
# Split en and zh
texts_en = list()
texts_zh = list()
for elm in train_ds:
    texts_en.append(elm['translation']['en'].numpy().decode("utf-8"))
    texts_zh.append(elm['translation']['zh'].numpy().decode("utf-8"))

print("Lists created")
print("%s%s" % ("Size: ", train_ds.cardinality().numpy()))

Lists created
Size: 231266


tokenizer stuff

In [35]:
sample_txt_en = texts_en[0]
sample_txt_zh = texts_zh[0]
print(sample_txt_en)
print(sample_txt_zh)

>>cmn<< We experience the world through a sequence of patterns, and we store them, and we recall them. And when we recall them, we match them up against reality, and we're making predictions all the time.
我们通过一系列的模式来感受世界环境，然后贮存， 再回想，当我们回想时，我们会比较和对应 实际情况，就这样我们不断的推测


In [65]:
from transformers import MarianTokenizer, MarianModel

model_name = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
print(tokenizer.supported_language_codes)

[]


C:\Users\kelvi\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [50]:
model = MarianModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(sample_txt_en, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

NameError: name 'TFMarianMTModel' is not defined

In [44]:
print(sample_txt_en)
print(str(tokenizer.tokenize(sample_txt_en)))
print(sample_txt_zh)
print(str(tokenizer.tokenize(sample_txt_zh)))

>>cmn<< We experience the world through a sequence of patterns, and we store them, and we recall them. And when we recall them, we match them up against reality, and we're making predictions all the time.
['>>cmn<<', '▁We', '▁experience', '▁the', '▁world', '▁through', '▁a', '▁sequence', '▁of', '▁patterns', ',', '▁and', '▁we', '▁store', '▁them', ',', '▁and', '▁we', '▁recall', '▁them', '.', '▁And', '▁when', '▁we', '▁recall', '▁them', ',', '▁we', '▁match', '▁them', '▁up', '▁against', '▁reality', ',', '▁and', '▁we', "'", 're', '▁making', '▁prediction', 's', '▁all', '▁the', '▁time', '.']
我们通过一系列的模式来感受世界环境，然后贮存， 再回想，当我们回想时，我们会比较和对应 实际情况，就这样我们不断的推测
['▁', '我们通过一系列的模式来感受世界环境', ',', '然后贮存', ',', '▁', '再回想', ',', '当我们回想时', ',', '我们会比较和对应', '▁', '实际情况', ',', '就这样我们不断的推测']


In [ ]:
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [ ]:
def lstm(input_shape):

    X_indices = Input(input_shape)

    embeddings = Embedding(input_dim = vocab_size, output_dim = embed_vector_len,
                           input_length = maxLen, weights = [embedding_matrix], trainable = False)(X_indices)

    X = LSTM(128)(embeddings)

    # X = Dropout(0.6)(X)

    # X = LSTM(128, return_sequences = True)(X)

    # X = Dropout(0.6)(X)

    # X = LSTM(128)(X)

    X = Dense(4, activation = 'softmax')(X)

    model = Model(inputs = X_indices, outputs = X)

    return model


Experimental

In [ ]:
from sklearn.model_selection import train_test_split

zh_tokenizer = tokenizer
en_tokenizer = tokenizer

zh_length = len(sample_txt_zh)
en_length = len(sample_txt_en)

# prepare training data
trainX = encode_sequences(zh_tokenizer, zh_length, train_ds[:, 1])
trainY = encode_sequences(en_tokenizer, en_length, train_ds[:, 0])

# prepare validation data
testX = encode_sequences(zh_tokenizer, zh_length, test_ds[:, 1])
testY = encode_sequences(en_tokenizer, en_length, test_ds[:, 0])

In [ ]:
# build NMT model
def define_model(in_vocab,out_vocab, in_timesteps,out_timesteps,units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dense(out_vocab, activation='softmax'))
    return model

In [ ]:
# model compilation
model = define_model(zh_vocab_size, en_vocab_size, zh_length, en_length, 512)

rms = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

In [ ]:
filename = 'iwslt-en-zh-lstm-v3'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=30, batch_size=512, validation_split = 0.2,callbacks=[checkpoint],
                    verbose=1)

plot result

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.show()

Ignore

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)

model = lstm((maxLen,   ))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train the model on the training data
epochs = 100
batch_size = 64

callback = EarlyStopping(monitor='loss', patience=3)
history = model.fit(X, Y, epochs = epochs, batch_size = batch_size, validation_split = 0.1, callbacks = [callback])